# 制約条件とペナルティ

## 制約条件付き最適化問題

数学的最適化において、制約とは解が満たさなければならない条件のことです。例えば、次の問題は制約付き最適化問題です：

$$
\begin{aligned}
\text{Minimize} & \quad f(x) \\
\text{subject to} & \quad g(x) = 0
\end{aligned}
$$

ここで、$f$と$g$は決定変数$x$の関数です。条件$g(x) = 0$は等式制約と呼ばれます。$g(x) = 0$を満たすすべての$x$の集合は実行可能集合と呼ばれます。制約は$g(x) = 0$のような等式制約や、$g(x) \leq 0$のような不等式制約である場合があります。例えば、次の問題も制約付き最適化問題です：

$$
\begin{aligned}
\text{Minimize} & \quad f(x) \\
\text{subject to} & \quad g(x) \leq 0
\end{aligned}
$$

`jijmodeling`では、`Constraint`クラスを使用して等式制約と不等式制約の両方を記述できます。例えば、等式制約$\sum_i x_i = 1$は次のように表現できます：

In [ ]:
import jijmodeling as jm

N = jm.Placeholder("N")
x = jm.BinaryVar("x", shape=(N,))
n = jm.Element('n', belong_to=(0, N))

jm.Constraint("onehot", jm.sum(n, x[n]) == 1)

上記のコードでは、`jm.Constraint`の最初の引数として文字列"onehot"があることに注意してください。制約オブジェクトには名前と制約式があります。これらの名前はサンプリングフェーズで制約が満たされているかどうかを確認するために使用されます。制約式は、`==`、`<=`、または`>=`の3つの比較演算子のいずれかを使用するブール式でなければなりません。次のようにして、1つの問題に複数の制約を課すことができます：

In [ ]:
x = jm.BinaryVar("x", shape=(4,))
problem = jm.Problem("constraint_sample")
problem += jm.Constraint("c01", x[0] + x[1] <= 1)
problem += jm.Constraint("c12", x[1] + x[2] <= 1)
problem += jm.Constraint("c23", x[2] + x[3] >= 0)

:::{tip}

他の比較演算子（例えば`>`）や論理演算子はサポートされていません。

In [ ]:
x = jm.BinaryVar("x", shape=(4,))
jm.Constraint("unsupported", (x[0] + x[1] <= 1) | (x[1] + x[2] <= 1))

:::

### `forall`制約
しばしば制約は変数によってインデックス付けされます。例えば、次の問題は制約付き最適化問題です：

$$
\begin{aligned}
\text{Minimize} & \quad \sum_{i=0}^{N-1} \sum_{j=0}^{M-1} a_{ij} x_{ij} \\
\text{subject to} & \quad \sum_{j = 0}^{M - 1} x_{ij} = 1 \quad \forall i \in \left\{0, \ldots, N - 1\right\}
\end{aligned}
$$

このような$\forall$制約を表現するために、`Constraint`オブジェクトには`forall`オプションがあります。例えば、上記の問題は次のように表現できます：

In [ ]:
N = jm.Placeholder("N")
M = jm.Placeholder("M")
a = jm.Placeholder("a", ndim=2)
x = jm.BinaryVar("x", shape=(N, M))
i = jm.Element('i', belong_to=(0, N))
j = jm.Element('j', belong_to=(0, M))

problem = jm.Problem("forall_sample")
problem += jm.sum([i, j], a[i, j] * x[i, j])
problem += jm.Constraint("onehot", jm.sum(j, x[i, j]) == 1, forall=i)

## ペナルティとは

[ペナルティ法](https://en.wikipedia.org/wiki/Penalty_method)と[ラグランジュ乗数法](https://en.wikipedia.org/wiki/Lagrange_multiplier) は、制約付き最適化問題を無制約のものに変換するための最も一般的な方法です。ペナルティ法では、制約付き最適化問題は次のように表現されます：

$$
\begin{aligned}
\text{Minimize} & \quad f(x) \\
\text{subject to} & \quad g(x) = 0
\end{aligned}
$$

この問題は次の形式の無制約最適化問題に変換されます：

$$
\text{Minimize} \quad f(x) + \alpha p(x),
$$

ここで、$\alpha$（ペナルティ係数またはラグランジュ乗数）と$p(x)$（ペナルティ項）が重要な役割を果たします。通常、$p(x)$は$p(x) = g(x)^2$として選ばれます。$f(x) + \alpha p(x)$の最小値が見つかり、$p(x) = 0$を満たす場合、元の制約付き最適化問題の最小値が見つかります。ペナルティ$p(x)$が正の場合、ペナルティ係数$\alpha$の値を増やして最適化問題を再実行します。

一部のソルバーは無制約問題のみを受け入れることに注意が必要です。QUBOの「U」は「Unconstrained（無制約）」を意味します。これらのソルバーを使用して`jijmodeling`で定義された制約付き問題を解決するために、`jijmodeling-transpiler`またはJijZeptのトランスパイラーレイヤーによって、デフォルトのペナルティ項$p$を使用して無制約問題に変換されます。しかし、実際の最適化問題において$p$の選択は重要であり、`jijmodeling`は`CustomPenaltyTerm`を使用してペナルティ項をカスタマイズする方法を提供します。


### 制約条件をペナルティに変換する

制約をペナルティ項に変換する作業は、`jijmodeling`自体ではなく、トランスパイラーによって処理されます。ここでは、`jijmodeling-transpiler`が制約をペナルティ項に変換する方法を説明します。小さな問題を考えてみましょう：

$$
\begin{aligned}
\text{Minimize} & \quad \sum_{i=0}^{N-1} a_i x_i \\
\text{subject to} & \quad \sum_{i = 0}^{N - 1} x_i = 1
\end{aligned}
$$

`jijmodeling-transpiler`の動作を確認するために：

In [ ]:
import jijmodeling as jm

a = jm.Placeholder("a", ndim=1)
N = a.len_at(0, latex="N")

i = jm.Element('i', belong_to=(0, N))
x = jm.BinaryVar('x', shape=(N,))

problem = jm.Problem('translate_constraint_to_penalty')
problem += jm.sum(i, a[i]*x[i])
problem += jm.Constraint(
    'onehot',
    jm.sum(i, x[i]) == 1,
)
problem

この問題は次の形式の制約なし問題に変換されます：

$$
\text{Minimize} \quad \sum_{i=0}^{N-1} a_i x_i + \alpha \left(\sum_{i = 0}^{N - 1} x_i - 1\right)^2
$$

$a = [1, 2]$および$\alpha = 5$の場合は次のようになります：

In [ ]:
import jijmodeling_transpiler as jmt

instance_data = {
    "a": [1, 2],
}

compiled_model = jmt.core.compile_model(problem, instance_data)
pubo_builder = jmt.core.pubo.transpile_to_pubo(
    compiled_model,
    normalize=False  # Disable normalization for simplicity
)
qubo, constant = pubo_builder.get_qubo_dict(multipliers={ 'onehot': 5 })

結果は次の通りです：

```
qubo     = {(0, 0): -4.0, (1, 1): -3.0, (0, 1): 10.0}
constant = 5.0
```

なぜなら：

$$
\begin{aligned}
\sum_{i=0}^{N-1} a_i x_i + \alpha \left(\sum_{i = 0}^{N - 1} x_i - 1\right)^2
&= x_1 + 2 x_2 + 5 (x_1 + x_2 - 1)^2 \\
&= -4 x_1 - 3 x_2 + 10 x_1 x_2 + 5 \\
&= \begin{bmatrix}
    x_1 & x_2 \end{bmatrix}
\begin{bmatrix}
    -4 & 10 \\
     0 & -3
\end{bmatrix}
\begin{bmatrix}
    x_1 \\
    x_2
\end{bmatrix}
+ 5
\end{aligned}
$$

バイナリ変数の場合、$x_i^2 = x_i$であることを確認してください。

このインスタンス化フェーズは2つのフェーズに分かれています：

- `Problem`オブジェクトと`instance_data`を使用して`CompiledInstance`オブジェクトに変換します。
- `CompiledInstance`オブジェクトを乗数を使用してQUBOに変換します。

`CompiledInstance`は「解決すべきもの」を表し、実際のQUBO係数は「どのように解決するか」を表します。

:::{note}

`jijmodeling-transpiler`は、`get_qubo_dict`メソッドの`detail_parameters`オプションを使用して、各ペナルティ項の乗数を設定する方法を提供します。[拡張ラグランジュ法](https://en.wikipedia.org/wiki/Augmented_Lagrangian_method)のような他の緩和方法もサポートされています。詳細については[jijmodeling-transpilerのリファレンス](https://www.documentation.jijzept.com/docs/jijmodelingtranspiler/references/jijmodeling_transpiler/core/pubo/)をご覧ください。

:::

### `CustomPenaltyTerm`

制約をペナルティ項に変換する作業はトランスパイラーの責任ですが、最初から別のトランスパイラーを作成せずに独自のペナルティ項を使用したい場合があります。`jijmodeling`は`CustomPenaltyTerm`を使用してペナルティ項をカスタマイズする方法を提供します。ここでは、前の例と同じペナルティ項を定義するために`CustomPenaltyTerm`を使用する方法を説明します：

$$
\text{Minimize} \quad \sum_{i=0}^{N-1} a_i x_i + \alpha \left(\sum_{i = 0}^{N - 1} x_i - 1\right)^2
$$

この問題は次のように`CustomPenaltyTerm`を使用して表現できます：

In [ ]:
import jijmodeling as jm

a = jm.Placeholder("a", ndim=1)
N = a.len_at(0, latex="N")

i = jm.Element('i', belong_to=(0, N))
x = jm.BinaryVar('x', shape=(N,))

problem = jm.Problem('penalty_sample')
problem += jm.sum(i, a[i]*x[i])
problem += jm.CustomPenaltyTerm(
    'onehot',
    (jm.sum(i, x[i]) - 1)**2,
)

ここでは乗数$\alpha$は表示されていないことに注意してください。これは前の例と同じ方法でインスタンス化できます：

In [ ]:
import jijmodeling_transpiler as jmt

instance_data = {
    "a": [1, 2],
}

compiled_model = jmt.core.compile_model(problem, instance_data)
pubo_builder = jmt.core.pubo.transpile_to_pubo(
    compiled_model,
    normalize=False  # Disable normalization for simplicity
)
qubo, constant = pubo_builder.get_qubo_dict(multipliers={ 'onehot': 5 })

結果は同じです：

```
qubo     = {(0, 0): -4.0, (1, 1): -3.0, (0, 1): 10.0}
constant = 5.0
```

`CustomPenaltyTerm`には`forall`オプションもあります：

In [1]:
import jijmodeling as jm

a = jm.Placeholder("a", ndim=2)
N = a.len_at(0, latex="N")
M = a.len_at(1, latex="M")

i = jm.Element('i', belong_to=(0, N))
j = jm.Element('j', belong_to=(0, M))
x = jm.BinaryVar('x', shape=(N, M))

problem = jm.Problem('forall_penalty_sample')
problem += jm.sum([i, j], a[i, j]*x[i, j])
problem += jm.CustomPenaltyTerm(
    'onehot',
    (jm.sum(j, x[i, j]) - 1)**2,
    forall=i
)
problem